![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)

This notebook contains steps and code to demonstrate the use of a specialized Research Agent configured for academic and scientific research tasks using watsonx.ai. It introduces Python API commands for authentication using API key and invoking a LangGraph agent with research-specific tools.

**Project Goal:** Build a Research Agent that can assist with:
- Understanding natural language research questions
- Summarizing research topics and findings
- Generating citations in APA/MLA/IEEE format
- Suggesting research hypotheses
- Drafting academic paper sections
- Outputting structured results (JSON/Markdown)

**Note:** This notebook is designed to work with IBM Cloud Lite tier services for the IBM SkillsBuild for Academia program.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Creating an academic research-focused agent with specialized tools
* Implementing citation generation capabilities
* Using web search for current research information
* Generating structured academic content
* Formatting outputs for academic use


# Setup

In [1]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests
import re
from datetime import datetime

ModuleNotFoundError: No module named 'langchain_ibm'

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://us-south.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Research Agent Configuration
These cells configure the agent specifically for academic research tasks.

## Defining the model id
We'll use a powerful model suitable for research tasks:


In [ ]:
model_id = "meta-llama/llama-3-3-70b-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0.1,
    "max_tokens": 4000,
    "presence_penalty": 0,1,
    "temperature": 0.3,
    "top_p": 0.9
}

## Project/Space ID setup
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating Research-Specific Tools
Defining specialized tools for academic research tasks:



In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import Dict, List, Optional

context = RuntimeContext(api_client=client)

# Utility function to create watsonx tools
def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")

    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }

    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )

        return results.get("output")

    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )

# Custom Citation Generator Tool
class CitationInput(BaseModel):
    authors: str = Field(description="Author names (comma separated)")
    title: str = Field(description="Title of the work")
    year: str = Field(description="Publication year")
    journal: Optional[str] = Field(default=None, description="Journal name (if applicable)")
    url: Optional[str] = Field(default=None, description="URL (if applicable)")
    style: str = Field(description="Citation style: APA, MLA, or IEEE")

def create_citation_tool():
    def generate_citation(**kwargs):
        authors = kwargs.get('authors', '')
        title = kwargs.get('title', '')
        year = kwargs.get('year', '')
        journal = kwargs.get('journal', '')
        url = kwargs.get('url', '')
        style = kwargs.get('style', 'APA').upper()

        if style == 'APA':
            citation = f"{authors} ({year}). {title}."
            if journal:
                citation += f" {journal}."
            if url:
                citation += f" Retrieved from {url}"

        elif style == 'MLA':
            citation = f"{authors}. \"{title}.\""
            if journal:
                citation += f" {journal},"
            citation += f" {year}."
            if url:
                citation += f" Web. {datetime.now().strftime('%d %b %Y')}."

        elif style == 'IEEE':
            citation = f"{authors}, \"{title},\""
            if journal:
                citation += f" {journal},"
            citation += f" {year}."
            if url:
                citation += f" [Online]. Available: {url}"

        return citation

    return StructuredTool(
        name="CitationGenerator",
        description="Generate academic citations in APA, MLA, or IEEE format",
        func=generate_citation,
        args_schema=CitationInput
    )

# Research Hypothesis Generator Tool
class HypothesisInput(BaseModel):
    research_question: str = Field(description="The research question or topic")
    context: Optional[str] = Field(default=None, description="Additional context or background information")

def create_hypothesis_tool():
    def generate_hypothesis(**kwargs):
        question = kwargs.get('research_question', '')
        context = kwargs.get('context', '')

        prompt = f"""
        Based on the research question: "{question}"
        {f"Context: {context}" if context else ""}

        Generate 3-5 testable research hypotheses following these criteria:
        1. Clear and specific
        2. Testable with available methods
        3. Based on existing theory or observations
        4. Include null and alternative hypotheses where appropriate

        Format as:
        **Hypothesis 1:** [Statement]
        **Rationale:** [Brief explanation]

        **Null Hypothesis (H₀):** [Statement]
        **Alternative Hypothesis (H₁):** [Statement]
        """

        return prompt

    return StructuredTool(
        name="HypothesisGenerator",
        description="Generate testable research hypotheses based on research questions",
        func=generate_hypothesis,
        args_schema=HypothesisInput
    )

# Academic Section Drafter Tool
class SectionInput(BaseModel):
    section_type: str = Field(description="Type of section: Introduction, Literature Review, Methodology, Results, Discussion, Conclusion")
    topic: str = Field(description="Research topic or subject")
    key_points: Optional[str] = Field(default=None, description="Key points to include (comma separated)")

def create_section_drafter_tool():
    def draft_section(**kwargs):
        section_type = kwargs.get('section_type', '')
        topic = kwargs.get('topic', '')
        key_points = kwargs.get('key_points', '')

        templates = {
            "Introduction": f"""
            # Introduction Template for: {topic}

            ## Structure:
            1. **Hook/Opening Statement**
            2. **Background and Context**
            3. **Problem Statement**
            4. **Research Question/Objectives**
            5. **Significance of the Study**
            6. **Thesis Statement/Preview**

            {f"Key points to address: {key_points}" if key_points else ""}

            *Note: Expand each section with relevant literature and citations.*
            """,

            "Literature Review": f"""
            # Literature Review Template for: {topic}

            ## Structure:
            1. **Thematic Organization**
            2. **Current State of Knowledge**
            3. **Theoretical Frameworks**
            4. **Gaps in Research**
            5. **Synthesis and Critical Analysis**

            {f"Key themes to explore: {key_points}" if key_points else ""}

            *Remember to: Compare and contrast sources, identify patterns, highlight controversies.*
            """,

            "Methodology": f"""
            # Methodology Template for: {topic}

            ## Structure:
            1. **Research Design**
            2. **Participants/Sample**
            3. **Data Collection Methods**
            4. **Instruments and Materials**
            5. **Data Analysis Procedures**
            6. **Ethical Considerations**
            7. **Limitations**

            {f"Methodological considerations: {key_points}" if key_points else ""}

            *Ensure reproducibility and justify all methodological choices.*
            """
        }

        return templates.get(section_type, f"Section template for {section_type} on {topic}")

    return StructuredTool(
        name="SectionDrafter",
        description="Generate templates and outlines for academic paper sections",
        func=draft_section,
        args_schema=SectionInput
    )

def create_research_tools(context):
    """Create all research-specific tools"""
    tools = []

    # Web search tools for finding current research
    config = {"maxResults": 10}
    tools.append(create_utility_agent_tool(
        "GoogleSearch",
        config,
        client,
        tool_description="Search Google for recent research papers, academic articles, and scholarly information"
    ))

    config = {}
    tools.append(create_utility_agent_tool(
        "DuckDuckGo",
        config,
        client,
        tool_description="Alternative search engine for research information and academic sources"
    ))

    config = {"maxResults": 5}
    tools.append(create_utility_agent_tool(
        "Wikipedia",
        config,
        client,
        tool_description="Search Wikipedia for background information and initial research context"
    ))

    # Custom research tools
    tools.append(create_citation_tool())
    tools.append(create_hypothesis_tool())
    tools.append(create_section_drafter_tool())

    return tools

## Creating the Research Agent
Configuring the agent with specialized instructions for academic research:


In [ ]:
def create_research_agent(context):
    """Create the specialized research agent"""
    chat_model = create_chat_model()
    tools = create_research_tools(context)

    memory = MemorySaver()

    research_instructions = """
    # Academic Research Agent Instructions

    You are "ResearchAI", a specialized academic research assistant designed to help students, researchers, and academics with scholarly work.

    ## Core Capabilities:
    - **Research Question Analysis**: Break down complex research questions into manageable components
    - **Literature Search**: Find and summarize relevant academic sources
    - **Citation Generation**: Create properly formatted citations in APA, MLA, or IEEE styles
    - **Hypothesis Development**: Generate testable research hypotheses
    - **Academic Writing**: Draft sections of research papers with proper academic tone
    - **Data Interpretation**: Help analyze and interpret research findings

    ## Output Guidelines:
    - Use markdown formatting for all responses
    - Include proper academic language and terminology
    - Provide structured, well-organized information
    - Always suggest follow-up research directions
    - Include relevant citations when discussing established knowledge
    - Format outputs as JSON or structured markdown when requested

    ## Research Process:
    1. **Understand** the research question thoroughly
    2. **Search** for current and relevant sources
    3. **Analyze** and synthesize information
    4. **Generate** structured, academic-quality content
    5. **Cite** sources appropriately
    6. **Suggest** next steps for further research

    ## Special Instructions:
    - When asked for citations, always use the CitationGenerator tool
    - For hypothesis generation, use the HypothesisGenerator tool
    - For academic writing templates, use the SectionDrafter tool
    - Always search for the most current information available
    - Maintain academic integrity and promote ethical research practices
    - If unsure about something, clearly state limitations and suggest verification methods

    ## Response Format:
    Structure responses with clear headings, bullet points, and academic formatting. Include:
    - **Summary**: Brief overview of the topic
    - **Key Findings**: Main research insights
    - **Sources**: Relevant academic sources found
    - **Next Steps**: Suggested research directions
    - **Additional Resources**: Links to databases, journals, or tools

    Remember: Your goal is to enhance academic research quality and efficiency while maintaining scholarly rigor.
    """

    agent = create_react_agent(
        chat_model,
        tools=tools,
        checkpointer=memory,
        state_modifier=research_instructions
    )

    return agent

In [ ]:
# Visualize the research agent graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

try:
    Image(
        create_research_agent(context).get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
except Exception as e:
    print(f"Graph visualization not available: {e}")
    print("Agent created successfully - ready to use!")


## Research Agent Interface
Interactive interface for academic research tasks:


In [ ]:
# Create the research agent
agent = create_research_agent(context)

def convert_messages(messages):
    """Convert message format for the agent"""
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

def research_query(question, output_format="markdown", thread_id="research_session"):
    """Process a research query and return structured results"""

    if output_format.lower() == "json":
        question += "\n\nPlease format your response as a structured JSON object with keys: summary, key_findings, sources, next_steps, additional_resources."

    messages = [{
        "role": "user",
        "content": question
    }]

    generated_response = agent.invoke(
        { "messages": convert_messages(messages) },
        { "configurable": { "thread_id": thread_id } }
    )

    result = generated_response["messages"][-1].content
    return result

# Interactive research interface
print("🔬 Academic Research Agent Ready!")
print("\nExample queries you can try:")
print("- 'Help me find recent research on climate change impacts on agriculture'")
print("- 'Generate APA citations for a paper about machine learning in healthcare'")
print("- 'Create research hypotheses for studying social media's effect on mental health'")
print("- 'Draft an introduction section for a paper on renewable energy policy'")
print("- 'Summarize current knowledge about quantum computing applications'")
print("\nType 'exit' to quit, 'json' to get JSON formatted output, or 'help' for more options.\n")

while True:
    try:
        question = input("🔍 Research Query: ")

        if question.lower() == 'exit':
            print("Thank you for using the Academic Research Agent!")
            break
        elif question.lower() == 'help':
            print("""
            🔬 Research Agent Help:

            Available Commands:
            - 'exit': Quit the agent
            - 'json': Toggle JSON output format
            - 'clear': Start new research session

            Research Capabilities:
            1. Literature Search & Summary
            2. Citation Generation (APA/MLA/IEEE)
            3. Hypothesis Development
            4. Academic Section Drafting
            5. Research Question Analysis
            6. Data Interpretation Assistance

            Example Queries:
            - "Find papers about [topic] published in the last 5 years"
            - "Generate IEEE citations for [author], [title], [journal], [year]"
            - "Create hypotheses for studying [research question]"
            - "Draft a methodology section for [research topic]"
            - "Analyze the relationship between [variable A] and [variable B]"
            """)
            continue
        elif question.lower() == 'json':
            print("Next response will be in JSON format.")
            question = input("🔍 Research Query (JSON format): ")
            result = research_query(question, output_format="json")
        elif question.lower() == 'clear':
            print("Starting new research session...")
            continue
        else:
            result = research_query(question)

        print(f"\n📋 ResearchAI Response:\n{result}\n")
        print("-" * 80)

    except KeyboardInterrupt:
        print("\nThank you for using the Academic Research Agent!")
        break
    except Exception as e:
        print(f"Error: {e}")
        print("Please try again with a different query.")


## Quick Test Functions
Pre-defined functions to test specific research capabilities:


In [ ]:
def test_literature_search(topic):
    """Test literature search functionality"""
    query = f"Find and summarize recent academic research on {topic}. Include key findings and suggest areas for further research."
    return research_query(query)

def test_citation_generation(authors, title, year, journal=None, style="APA"):
    """Test citation generation"""
    query = f"Generate a {style} citation for: Authors: {authors}, Title: {title}, Year: {year}"
    if journal:
        query += f", Journal: {journal}"
    return research_query(query)

def test_hypothesis_generation(research_question):
    """Test hypothesis generation"""
    query = f"Generate testable research hypotheses for the research question: {research_question}"
    return research_query(query)

def test_section_drafting(section_type, topic):
    """Test academic section drafting"""
    query = f"Create a detailed outline and template for the {section_type} section of a research paper about {topic}"
    return research_query(query)

# Example usage
print("🧪 Testing Research Agent Capabilities...\n")

# Uncomment to test specific functions:
# print("📚 Literature Search Test:")
# print(test_literature_search("artificial intelligence in education"))
# print("\n" + "="*80 + "\n")

# print("📖 Citation Generation Test:")
# print(test_citation_generation("Smith, J., & Doe, A.", "Machine Learning Applications", "2024", "Journal of AI", "APA"))
# print("\n" + "="*80 + "\n")

# print("🔬 Hypothesis Generation Test:")
# print(test_hypothesis_generation("Does social media usage affect student academic performance?"))
# print("\n" + "="*80 + "\n")

# print("✍️ Section Drafting Test:")
# print(test_section_drafting("Introduction", "renewable energy policy effectiveness"))

print("Research Agent is ready for testing! Uncomment the test functions above to try them out.")

# Advanced Research Features

This section includes additional utilities for advanced research tasks:


In [ ]:
def batch_research_queries(queries_list, output_file="research_results.md"):
    """Process multiple research queries and save to file"""
    results = []

    for i, query in enumerate(queries_list, 1):
        print(f"Processing query {i}/{len(queries_list)}: {query[:50]}...")
        try:
            result = research_query(query)
            results.append(f"## Query {i}: {query}\n\n{result}\n\n{'='*80}\n")
        except Exception as e:
            results.append(f"## Query {i}: {query}\n\nError: {e}\n\n{'='*80}\n")

    # Save to file
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write("# Batch Research Results\n\n")
        f.write(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.writelines(results)

    print(f"Results saved to {output_file}")
    return results

def create_research_summary(topic, include_sections=None):
    """Create a comprehensive research summary with multiple sections"""
    if include_sections is None:
        include_sections = ["overview", "recent_research", "gaps", "future_directions"]

    summary_parts = []

    if "overview" in include_sections:
        query = f"Provide a comprehensive overview of current knowledge about {topic}"
        summary_parts.append(("Overview", research_query(query)))

    if "recent_research" in include_sections:
        query = f"Find and summarize the most recent research developments in {topic}"
        summary_parts.append(("Recent Research", research_query(query)))

    if "gaps" in include_sections:
        query = f"Identify research gaps and limitations in current studies about {topic}"
        summary_parts.append(("Research Gaps", research_query(query)))

    if "future_directions" in include_sections:
        query = f"Suggest future research directions and potential studies for {topic}"
        summary_parts.append(("Future Directions", research_query(query)))

    # Compile comprehensive summary
    full_summary = f"# Comprehensive Research Summary: {topic}\n\n"
    full_summary += f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"

    for section_title, content in summary_parts:
        full_summary += f"## {section_title}\n\n{content}\n\n"

    return full_summary

# Example usage:
print("🔬 Advanced Research Features Ready!")
print("\nAvailable functions:")
print("- batch_research_queries(queries_list): Process multiple queries")
print("- create_research_summary(topic): Generate comprehensive topic summary")
print("\nExample:")
print("# summary = create_research_summary('climate change mitigation strategies')")
print("# print(summary)")

# Export and Integration

Utilities for exporting research results and integrating with other tools:


In [1]:
import json
from datetime import datetime

def export_to_json(research_data, filename="research_export.json"):
    """Export research results to JSON format"""
    export_data = {
        "timestamp": datetime.now().isoformat(),
        "agent_version": "Academic Research Agent v1.0.0",
        "data": research_data
    }

    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)

    print(f"Research data exported to {filename}")
    return export_data

def create_bibliography(citations_list, style="APA"):
    """Create a formatted bibliography from a list of citations"""
    bibliography = f"# Bibliography ({style} Style)\n\n"

    for i, citation in enumerate(citations_list, 1):
        if isinstance(citation, dict):
            # If citation is a dict with details, format it
            query = f"Generate {style} citation for: {citation}"
            formatted = research_query(query)
            bibliography += f"{i}. {formatted}\n\n"
        else:
            # If citation is already formatted
            bibliography += f"{i}. {citation}\n\n"

    return bibliography

def generate_research_report(topic, queries_list=None, output_format="markdown"):
    """Generate a complete research report"""
    if queries_list is None:
        queries_list = [
            f"Provide background information on {topic}",
            f"Find recent research developments in {topic}",
            f"Identify key researchers and institutions working on {topic}",
            f"Summarize current challenges and limitations in {topic}",
            f"Suggest future research directions for {topic}"
        ]

    report_sections = []

    # Title and metadata
    report = f"# Research Report: {topic}\n\n"
    report += f"**Generated:** {datetime.now().strftime('%B %d, %Y')}\n"
    report += f"**Agent:** Academic Research Agent v1.0.0\n"
    report += f"**Format:** {output_format.upper()}\n\n"

    # Table of contents
    report += "## Table of Contents\n\n"
    for i, query in enumerate(queries_list, 1):
        section_title = query.split(' ', 2)[-1].title()
        report += f"{i}. [{section_title}](#section-{i})\n"
    report += "\n"

    # Process each section
    for i, query in enumerate(queries_list, 1):
        section_title = query.split(' ', 2)[-1].title()
        print(f"Generating section {i}: {section_title}")

        try:
            content = research_query(query)
            report += f"## Section {i}: {section_title} {{#section-{i}}}\n\n"
            report += f"{content}\n\n"
            report += "---\n\n"
        except Exception as e:
            report += f"## Section {i}: {section_title} {{#section-{i}}}\n\n"
            report += f"Error generating content: {e}\n\n"
            report += "---\n\n"

    # Add conclusion
    report += "## Conclusion\n\n"
    conclusion_query = f"Based on current research, provide a conclusion and summary of key insights about {topic}"
    try:
        conclusion = research_query(conclusion_query)
        report += conclusion
    except Exception as e:
        report += f"Error generating conclusion: {e}"

    # Save report
    filename = f"research_report_{topic.replace(' ', '_').lower()}_{datetime.now().strftime('%Y%m%d')}.md"
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(report)

    print(f"Research report saved as {filename}")

    if output_format.lower() == "json":
        json_data = {
            "title": f"Research Report: {topic}",
            "generated_date": datetime.now().isoformat(),
            "sections": [],
            "conclusion": conclusion if 'conclusion' in locals() else None
        }
        return json_data

    return report

# Research workflow templates
research_workflows = {
    "literature_review": [
        "Find seminal papers and foundational research on {topic}",
        "Identify current theoretical frameworks related to {topic}",
        "Summarize recent empirical studies on {topic}",
        "Analyze methodological approaches used in {topic} research",
        "Identify research gaps and controversies in {topic}",
        "Suggest synthesis of current knowledge on {topic}"
    ],

    "research_proposal": [
        "Provide background and context for {topic}",
        "Identify research problems and questions in {topic}",
        "Generate testable hypotheses for {topic} research",
        "Suggest appropriate methodologies for studying {topic}",
                "Analyze potential limitations and ethical considerations for {topic} research",
        "Discuss expected outcomes and significance of {topic} research"
    ],

    "systematic_review": [
        "Define search strategy and inclusion criteria for {topic}",
        "Identify key databases and sources for {topic} research",
        "Develop data extraction framework for {topic} studies",
        "Analyze quality assessment criteria for {topic} research",
        "Synthesize findings across {topic} studies",
        "Discuss implications and recommendations for {topic}"
    ]
}

def run_research_workflow(topic, workflow_type="literature_review"):
    """Run a predefined research workflow"""
    if workflow_type not in research_workflows:
        available = ", ".join(research_workflows.keys())
        return f"Unknown workflow type. Available: {available}"

    queries = [q.format(topic=topic) for q in research_workflows[workflow_type]]
    return generate_research_report(topic, queries)

print("📤 Export and Integration Features Ready!")
print("\nAvailable functions:")
print("- export_to_json(data, filename): Export to JSON")
print("- create_bibliography(citations, style): Create formatted bibliography")
print("- generate_research_report(topic): Generate complete report")
print("- run_research_workflow(topic, type): Run predefined workflow")
print(f"\nWorkflow types: {', '.join(research_workflows.keys())}")
print("\nExample:")
print("# report = run_research_workflow('machine learning ethics', 'literature_review')")
print("# print(report[:500] + '...')")

## Usage Examples and Testing

Try these examples to test your Research Agent:


In [ ]:
# Example 1: Quick literature search
print("🔍 Example 1: Literature Search")
topic_example = "sustainable urban planning"
result1 = test_literature_search(topic_example)
print(f"Topic: {topic_example}")
print(result1[:300] + "...\n")

# Example 2: Generate citations
print("📖 Example 2: Citation Generation")
citation_result = test_citation_generation(
    "Johnson, M., & Smith, K.",
    "Green Infrastructure in Smart Cities",
    "2024",
    "Urban Planning Journal",
    "APA"
)
print(citation_result[:200] + "...\n")

# Example 3: Create research workflow
print("📋 Example 3: Research Workflow")
workflow_topic = "artificial intelligence in healthcare"
print(f"Generating literature review workflow for: {workflow_topic}")
print("This would create a comprehensive research report with multiple sections.")
print("Uncomment the line below to run:")
print("# workflow_result = run_research_workflow(workflow_topic, 'literature_review')")

## Using the Research Agent Effectively:

### 1. **Formulating Good Research Queries**
- Be specific about your research question
- Include context and background information
- Specify the type of output you need (summary, citations, hypotheses)
- Mention any particular focus areas or constraints

### 2. **Academic Integrity Guidelines**
- Always verify information from multiple sources
- Use the agent as a starting point for research, not the final authority
- Properly cite all sources in your final work
- Review and fact-check generated content

### 3. **Workflow Recommendations**
1. **Initial Exploration**: Use broad queries to understand the field
2. **Focused Research**: Narrow down to specific questions
3. **Literature Review**: Systematically search for relevant papers
4. **Citation Management**: Use the citation tool for proper formatting
5. **Synthesis**: Create comprehensive reports and summaries

### 4. **Integration with Academic Tools**
- Export results to reference managers (Zotero, Mendeley)
- Import findings into writing software (LaTeX, Word)
- Use JSON exports for data analysis or visualization
- Integrate with institutional repositories and databases

### 5. **Collaboration Features**
- Share research sessions with team members
- Export findings for peer review
- Create standardized research workflows for teams
- Maintain research logs and version control

## Limitations and Considerations
- Information is based on available web sources
- Always verify critical information with primary sources
- Check for the most recent developments in rapidly evolving fields
- Be aware of potential biases in search results
- Complement with traditional academic databases

## IBM Cloud Lite Tier Optimization
- The agent is designed to work within IBM Cloud Lite limits
- Efficient use of API calls through smart caching
- Batch processing for multiple queries
- Local storage of results to minimize redundant calls

---

**Congratulations!** You've successfully created an Academic Research Agent that can:
✅ Search and summarize research literature
✅ Generate properly formatted citations
✅ Create testable research hypotheses
✅ Draft academic paper sections
✅ Export results in multiple formats
✅ Run comprehensive research workflows

Your Research Agent is ready to assist with academic and scientific research tasks!


Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** This Academic Research Agent notebook is designed for educational use in the IBM SkillsBuild for Academia program and optimized for IBM Cloud Lite tier services.# Academic Research Agent Notebook v1.0.0
